In [1]:
import pandas as pd
import numpy as np

1\. **Text files**

Perform the following operations on plain `txt` files:

+ create a list of integrer numbers and then save it to a text file named `data_int.txt`. Run the `cat` command to print the content of the file.
+ create a matrix of 5x5 floats and then save it to a text file named `data_float.txt`. Use the `cat` command to print the content of the file.
+ load the `txt` file of the previous point and convert it to a `csv` file by hand.

In [2]:
import numpy.random as npr
import csv

#text file name
file_name1 = './data/data_int.txt'
file_name2 = './data/data_float.txt'

#list of integer numbers
list_int = np.arange(10)
#save it in a .txt file
np.savetxt(file_name1, list_int)
#read it with the cat command
!cat data/data_int.txt

#a matrix of 5x5 floats
a =  npr.uniform(size=(5,5), low = 0, high = 10)
#save it in a .txt file
np.savetxt(file_name2, a, fmt='%.2e')
print("\n")
#read it with the command cat
!cat data/data_float.txt

#we load the previous file
with open(file_name2, 'r') as  in_file, open('./data/data_float.csv', 'w') as out_file:
    #we remove the leading and the trailing characters
    stripped = [line.strip() for line in in_file]
    stripped_bis = []
    # separating values belonging to a same line by ';' whereas ' ' (csv format)
    for s in stripped:
        s = s.replace(' ',';')
        stripped_bis.append(s)
    lines = (line.split(",") for line in stripped_bis if line)
    #we write the information in a csv file
    writer = csv.writer(out_file)
    writer.writerows(lines)   

#The easy way to do it but not by hands
#np.savetxt('./data/data_float.csv', a, fmt='%.2e', delimiter=',')


0.000000000000000000e+00
1.000000000000000000e+00
2.000000000000000000e+00
3.000000000000000000e+00
4.000000000000000000e+00
5.000000000000000000e+00
6.000000000000000000e+00
7.000000000000000000e+00
8.000000000000000000e+00
9.000000000000000000e+00


3.84e+00 3.88e+00 5.80e+00 6.35e-01 9.34e+00
5.16e+00 4.14e+00 3.99e-01 4.55e+00 6.33e+00
8.67e+00 8.62e+00 6.40e+00 2.42e+00 7.70e+00
6.37e+00 3.20e+00 3.41e+00 6.04e-01 3.79e+00
9.79e-01 5.01e+00 2.61e+00 3.67e+00 9.08e+00


2\. **JSON files**

Load the file *user_data.json*, which can be found at:

- https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json

and filter the data by the "CreditCardType" when it equals to "American Express". Than save the data to a new CSV file.

In [3]:
#to load the file user_data.json
#!wget https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json -P data/

#import json #import the JSON module
#data = json.load(open('data/user_data.json'))
#filter_json = [x for x in data if x['CreditCardType'] == 'American Express']

df = pd.read_json('data/user_data.json')
df = df.loc[df['CreditCardType'] == 'American Express']
df.to_csv('./data/user_data.csv', index= None)

3\. **CSV files with Pandas**

Load the file from this url:

- https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv

with Pandas. 

+ explore and print the DataFrame
+ calculate, using `groupby()`, the average value of each feature, separately for each class
+ save the file in a JSON format.

In [4]:
#!wget https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv -P data/
df = pd.read_csv('data/mushrooms_categorized.csv')
df_average = df.groupby(['class'])[df.columns.values[1:]].mean()
df_average.to_json('data/mushrooms_categorized.json',indent=4)

!cat data/mushrooms_categorized.json

{
    "cap-shape":{
        "0":3.2661596958,
        "1":3.4361593463
    },
    "cap-surface":{
        "0":1.6159695817,
        "1":2.0551583248
    },
    "cap-color":{
        "0":4.5817490494,
        "1":4.4218590398
    },
    "bruises":{
        "0":0.6539923954,
        "1":0.1593462717
    },
    "odor":{
        "0":4.3346007605,
        "1":3.9407558733
    },
    "gill-attachment":{
        "0":0.9543726236,
        "1":0.9954034729
    },
    "gill-spacing":{
        "0":0.2851711027,
        "1":0.0286006129
    },
    "gill-size":{
        "0":0.0684410646,
        "1":0.5679264556
    },
    "gill-color":{
        "0":6.6226235741,
        "1":2.8636363636
    },
    "stalk-shape":{
        "0":0.6159695817,
        "1":0.5148110317
    },
    "stalk-root":{
        "0":1.4980988593,
        "1":0.6925434116
    },
    "stalk-surface-above-ring":{
        "0":1.7756653992,
        "1":1.3595505618
    },
    "stalk-surface-below-ring":{
        "0":1.7984790875,
    

4\. **Reading the credit card numbers**

Get the binary file named *credit_card.dat* from this address:

- https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat

and convert the data into the real credit card number, knowing that:
- each line corresponds to a credit card number, which consists of 16 characters (which are numbers in the 0-9 range) divided in 4 blocks, with a whitespace between each block
- each character is written using a 6 bit binary representation (including the whitespace)
- the final 4 bits of each line are a padding used to determine the end of the line, and can be ignored

*Hint*: convert the binary numbers to the decimal representation first, and then use the `chr()` function to convert the latter to a char

In [10]:
#!wget https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat -P data/
data = {}
df = pd.DataFrame({})

with open('data/credit_card.dat', 'rb') as file:
    file_content = file.read()
    word_counter = 0
    #each line : we read a word of 8 bytes
    for i in range(0, len(file_content), 16):
        print(i)

0
16
32
48
64
80
96
112
128
144
160
176
192
208
224
240
256
272
288
304
320
336
352
368
384
400
416
432
448
464
480
496
512
528
544
560
576
592
608
624
640
656
672
688
704
720
736
752
768
784
800
816
832
848
864
880
896
912
928
944
960
976
992
1008
1024
1040
1056
1072
1088
1104
1120
1136
1152
1168
1184
1200
1216
1232
1248
1264
1280
1296
1312
1328
1344
1360
1376
1392
1408
1424
1440
1456
1472
1488
1504
1520
1536
1552
1568
1584
1600
1616
1632
1648
1664
1680
1696
1712
1728
1744
1760
1776
1792
1808
1824
1840
1856
1872
1888
1904
1920
1936
1952
1968
1984
2000
2016
2032
2048
2064
2080
2096
2112
2128
2144
2160
2176
2192
2208
2224
2240
2256
2272
2288
2304
2320
2336
2352
2368
2384
2400
2416
2432
2448
2464
2480
2496
2512
2528
2544
2560
2576
2592
2608
2624
2640
2656
2672
2688
2704
2720
2736
2752
2768
2784
2800
2816
2832
2848
2864
2880
2896
2912
2928
2944
2960
2976
2992
3008
3024
3040
3056
3072
3088
3104
3120
3136
3152
3168
3184
3200
3216
3232
3248
3264
3280
3296
3312
3328
3344
3360
3376
3392
3408
3

5\. **Write data to a binary file**

a) Start from the `data/data_000637.txt` file that we have used during the previous lectures, and convert it to a binary file according to the format defined below:

In [ ]:
from IPython.display import Image
Image("images/data_format.png")

*Hints*:
- Read the first 10 lines using Pandas
- Iterate over the DataFrame rows
- For every row, ``pack'' the values (features) into a single 64-bit word, according to the format specified above. Use bit-wise shifts and operators to do so.
- Write each 64-bit word to a binary file. You can use `struct` in this way:
```
binary_file.write( struct.pack('<q', word) )
```
where `word` is the 64-bit word.
- Close the file after completing the loop.

b) Check that the binary file is correctly written by reading it with the code used in the lecture `06_OSEMN.ipynb`, and verify that the content of the `txt` and binary files is consistent.

c) What is the difference of the size on disk between equivalent `txt` and binary files?